# Analytics Database Queries

This notebook demonstrates how to query the Spindle analytics SQLite database using both direct SQL queries and the AnalyticsStore API.

## Overview

The analytics database stores:
- **Document Observations**: Structured metrics about ingested documents
- **Service Events**: LLM calls, latency, costs, and other observability data

## Database Schema

- `ingestion_observations`: Main table storing document observations as JSON
- `ingestion_observation_events`: Service events linked to observations


In [1]:
from __future__ import annotations

import sqlite3
from pathlib import Path
from datetime import datetime
import pandas as pd
from pprint import pprint

# Spindle imports
from spindle.analytics import AnalyticsStore
from spindle.analytics.views import (
    corpus_overview,
    document_size_table,
    chunk_window_risk,
    observability_events,
    ontology_recommendation_metrics,
    triple_extraction_metrics,
    entity_resolution_metrics,
)

print("✅ Imports successful")


✅ Imports successful


## Step 1: Connect to the Analytics Database

First, we'll locate and connect to the analytics database. By default, it's stored at `{project_root}/logs/analytics.db`.


In [2]:
# Path to analytics database
# Update this path to point to your analytics database
ANALYTICS_DB_PATH = Path("example_project/logs/analytics.db")

# Check if database exists
if not ANALYTICS_DB_PATH.exists():
    print(f"⚠️  Database not found at: {ANALYTICS_DB_PATH}")
    print("   Please update ANALYTICS_DB_PATH to point to your analytics database")
    print("   Or run the example_e2e.ipynb notebook first to generate sample data")
else:
    print(f"✅ Found database at: {ANALYTICS_DB_PATH}")
    print(f"   Database size: {ANALYTICS_DB_PATH.stat().st_size / 1024:.2f} KB")

# Create AnalyticsStore instance
database_url = f"sqlite:///{ANALYTICS_DB_PATH.resolve()}"
store = AnalyticsStore(database_url)
print(f"✅ Connected to database: {database_url}")


✅ Found database at: example_project\logs\analytics.db
   Database size: 76.00 KB
✅ Connected to database: sqlite:///C:\Users\danie\Repos\spindle\spindle\notebooks\example_project\logs\analytics.db


## Step 2: Explore Database Schema

Let's examine the database structure using direct SQL queries.


In [3]:
events = store.fetch_service_events(service="ontology.recommender")

In [4]:
events[0].__dict__

{'timestamp': datetime.datetime(2025, 11, 16, 8, 0, 48, 839496),
 'service': 'ontology.recommender',
 'name': 'recommend.start',
 'payload': {'scope': 'balanced', 'text_length': 1788}}

In [5]:
events[1].__dict__

{'timestamp': datetime.datetime(2025, 11, 16, 8, 1, 50, 745960),
 'service': 'ontology.recommender',
 'name': 'recommend.complete',
 'payload': {'scope': 'balanced',
  'entity_type_count': 7,
  'relation_type_count': 15,
  'input_tokens': 2562,
  'output_tokens': 4276,
  'cost': 0.009193,
  'latency_ms': 61119.81400009245,
  'model': 'gpt-5-mini-2025-08-07'}}

In [9]:
store.fetch_service_events()

[ServiceEventRecord(timestamp=datetime.datetime(2025, 11, 16, 7, 11, 44, 655078), service='ingestion.service', name='run.start', payload={'path_count': 3, 'catalog_url': 'sqlite:///c:\\Users\\danie\\Repos\\spindle\\spindle\\notebooks\\example_project\\catalog\\ingestion.db', 'vector_store_uri': 'c:\\Users\\danie\\Repos\\spindle\\spindle\\notebooks\\example_project\\vector_store'}),
 ServiceEventRecord(timestamp=datetime.datetime(2025, 11, 16, 7, 11, 44, 677214), service='ingestion.pipeline', name='stage_start', payload={'stage': 'checksum'}),
 ServiceEventRecord(timestamp=datetime.datetime(2025, 11, 16, 7, 11, 44, 677214), service='ingestion.pipeline', name='stage_start', payload={'stage': 'checksum'}),
 ServiceEventRecord(timestamp=datetime.datetime(2025, 11, 16, 7, 11, 44, 680789), service='ingestion.pipeline', name='stage_complete', payload={'stage': 'checksum', 'duration_ms': 4.391300026327372}),
 ServiceEventRecord(timestamp=datetime.datetime(2025, 11, 16, 7, 11, 44, 680789), serv

In [6]:
store.close()

In [3]:
# Connect directly to SQLite for schema exploration
conn = sqlite3.connect(str(ANALYTICS_DB_PATH))

# List all tables
tables_query = "SELECT name FROM sqlite_master WHERE type='table'"
tables = pd.read_sql_query(tables_query, conn)
print("📊 Tables in database:")
print(tables)

# Get schema for each table
for table_name in tables['name']:
    print(f"\n📋 Schema for '{table_name}':")
    schema_query = f"PRAGMA table_info({table_name})"
    schema = pd.read_sql_query(schema_query, conn)
    print(schema.to_string(index=False))

conn.close()


📊 Tables in database:
                           name
0                service_events
1        ingestion_observations
2  ingestion_observation_events

📋 Schema for 'service_events':
 cid      name     type  notnull dflt_value  pk
   0        id  INTEGER        1       None   1
   1 timestamp DATETIME        1       None   0
   2   service  VARCHAR        1       None   0
   3      name  VARCHAR        1       None   0
   4   payload     JSON        1       None   0

📋 Schema for 'ingestion_observations':
 cid           name     type  notnull dflt_value  pk
   0             id  INTEGER        1       None   1
   1    document_id  VARCHAR        1       None   0
   2    ingested_at DATETIME        1       None   0
   3 schema_version  VARCHAR        1       None   0
   4        payload     JSON        1       None   0
   5     created_at DATETIME        1       None   0

📋 Schema for 'ingestion_observation_events':
 cid           name     type  notnull dflt_value  pk
   0             id 

## Step 3: Basic Queries Using AnalyticsStore API

The AnalyticsStore provides convenient methods for querying observations and events.


In [4]:
# Fetch all observations (limit to 10 for display)
observations = store.fetch_observations(limit=10)
print(f"📄 Found {len(observations)} observations (showing up to 10)")

if observations:
    print("\n📋 Sample observation:")
    obs = observations[0]
    print(f"  Document ID: {obs.metadata.document_id}")
    print(f"  Ingested at: {obs.metadata.ingested_at}")
    print(f"  Token count: {obs.structural.token_count}")
    print(f"  Chunk count: {obs.structural.chunk_count}")
    print(f"  Source URI: {obs.metadata.source_uri}")
    print(f"  Schema version: {obs.schema_version}")
else:
    print("⚠️  No observations found in database")


📄 Found 3 observations (showing up to 10)

📋 Sample observation:
  Document ID: eac54e99f2a94080964eafd9c6af1902
  Ingested at: 2025-11-14 22:18:11.425474
  Token count: 83
  Chunk count: 1
  Source URI: C:\Users\danie\Repos\spindle\spindle\notebooks\example_project\documents\doc3_techcorp_financials.md
  Schema version: 1.0.0


In [5]:
# Fetch service events
events = store.fetch_service_events(limit=20)
print(f"🔔 Found {len(events)} service events (showing up to 20)")

if events:
    print("\n📋 Sample events:")
    for i, event in enumerate(events[:5], 1):
        print(f"\n  Event {i}:")
        print(f"    Service: {event.service}")
        print(f"    Name: {event.name}")
        print(f"    Timestamp: {event.timestamp}")
        print(f"    Payload keys: {list(event.payload.keys())}")
else:
    print("⚠️  No service events found in database")


🔔 Found 20 service events (showing up to 20)

📋 Sample events:

  Event 1:
    Service: ingestion.service
    Name: run.start
    Timestamp: 2025-11-14 22:18:11.157642
    Payload keys: ['path_count', 'catalog_url', 'vector_store_uri']

  Event 2:
    Service: ingestion.pipeline
    Name: stage_start
    Timestamp: 2025-11-14 22:18:11.170897
    Payload keys: ['stage']

  Event 3:
    Service: ingestion.pipeline
    Name: stage_start
    Timestamp: 2025-11-14 22:18:11.170897
    Payload keys: ['stage']

  Event 4:
    Service: ingestion.pipeline
    Name: stage_complete
    Timestamp: 2025-11-14 22:18:11.184624
    Payload keys: ['stage', 'duration_ms']

  Event 5:
    Service: ingestion.pipeline
    Name: stage_complete
    Timestamp: 2025-11-14 22:18:11.184624
    Payload keys: ['stage', 'duration_ms']


## Step 4: Direct SQL Queries

Let's perform some direct SQL queries for more detailed analysis.


In [6]:
conn = sqlite3.connect(str(ANALYTICS_DB_PATH))

# Query 1: Count total observations
query1 = "SELECT COUNT(*) as total_observations FROM ingestion_observations"
result1 = pd.read_sql_query(query1, conn)
print("📊 Query 1: Total observations")
print(result1.to_string(index=False))

# Query 2: List all document IDs with ingestion timestamps
query2 = """
SELECT 
    document_id,
    ingested_at,
    schema_version,
    created_at
FROM ingestion_observations
ORDER BY ingested_at DESC
LIMIT 10
"""
result2 = pd.read_sql_query(query2, conn)
print("\n📊 Query 2: Recent document observations")
print(result2.to_string(index=False))

conn.close()


📊 Query 1: Total observations
 total_observations
                  3

📊 Query 2: Recent document observations
                     document_id                ingested_at schema_version                 created_at
eac54e99f2a94080964eafd9c6af1902 2025-11-14 22:18:11.425474          1.0.0 2025-11-14 22:18:11.919544
2a6b68c8b120469ea6cb21707ecbfeaf 2025-11-14 22:18:11.327198          1.0.0 2025-11-14 22:18:11.919544
3f857ca539ef4595ae6b2775c23f07ed 2025-11-14 22:18:11.233879          1.0.0 2025-11-14 22:18:11.914128


In [7]:
conn = sqlite3.connect(str(ANALYTICS_DB_PATH))

# Query 3: Count service events by service name
query3 = """
SELECT 
    service,
    COUNT(*) as event_count
FROM ingestion_observation_events
GROUP BY service
ORDER BY event_count DESC
"""
result3 = pd.read_sql_query(query3, conn)
print("📊 Query 3: Service events by service")
print(result3.to_string(index=False))

# Query 4: Count service events by event name
query4 = """
SELECT 
    name,
    COUNT(*) as event_count
FROM ingestion_observation_events
GROUP BY name
ORDER BY event_count DESC
LIMIT 10
"""
result4 = pd.read_sql_query(query4, conn)
print("\n📊 Query 4: Top event types")
print(result4.to_string(index=False))

conn.close()


📊 Query 3: Service events by service
           service  event_count
ingestion.pipeline            3

📊 Query 4: Top event types
       name  event_count
graph_built            3


In [8]:
conn = sqlite3.connect(str(ANALYTICS_DB_PATH))

# Query 5: Join observations with their events
query5 = """
SELECT 
    o.document_id,
    o.ingested_at,
    COUNT(e.id) as event_count,
    COUNT(DISTINCT e.service) as unique_services
FROM ingestion_observations o
LEFT JOIN ingestion_observation_events e ON o.id = e.observation_id
GROUP BY o.id, o.document_id, o.ingested_at
ORDER BY event_count DESC
LIMIT 10
"""
result5 = pd.read_sql_query(query5, conn)
print("📊 Query 5: Observations with event counts")
print(result5.to_string(index=False))

conn.close()


📊 Query 5: Observations with event counts
                     document_id                ingested_at  event_count  unique_services
3f857ca539ef4595ae6b2775c23f07ed 2025-11-14 22:18:11.233879            1                1
2a6b68c8b120469ea6cb21707ecbfeaf 2025-11-14 22:18:11.327198            1                1
eac54e99f2a94080964eafd9c6af1902 2025-11-14 22:18:11.425474            1                1


## Step 5: Extract JSON Data from Observations

The observation payloads are stored as JSON. Let's extract specific fields for analysis.


In [9]:
conn = sqlite3.connect(str(ANALYTICS_DB_PATH))

# Query 6: Extract token counts from JSON payload
query6 = """
SELECT 
    document_id,
    ingested_at,
    json_extract(payload, '$.structural.token_count') as token_count,
    json_extract(payload, '$.structural.chunk_count') as chunk_count,
    json_extract(payload, '$.metadata.source_uri') as source_uri,
    json_extract(payload, '$.metadata.source_type') as source_type
FROM ingestion_observations
ORDER BY token_count DESC
LIMIT 10
"""
result6 = pd.read_sql_query(query6, conn)
print("📊 Query 6: Document structural metrics")
print(result6.to_string(index=False))

conn.close()


📊 Query 6: Document structural metrics
                     document_id                ingested_at  token_count  chunk_count                                                                                           source_uri source_type
3f857ca539ef4595ae6b2775c23f07ed 2025-11-14 22:18:11.233879          242            4   C:\Users\danie\Repos\spindle\spindle\notebooks\example_project\documents\doc1_techcorp_overview.md        file
2a6b68c8b120469ea6cb21707ecbfeaf 2025-11-14 22:18:11.327198           92            1  C:\Users\danie\Repos\spindle\spindle\notebooks\example_project\documents\doc2_techcorp_expansion.md        file
eac54e99f2a94080964eafd9c6af1902 2025-11-14 22:18:11.425474           83            1 C:\Users\danie\Repos\spindle\spindle\notebooks\example_project\documents\doc3_techcorp_financials.md        file


In [10]:
conn = sqlite3.connect(str(ANALYTICS_DB_PATH))

# Query 7: Extract context strategy recommendations
query7 = """
SELECT 
    document_id,
    json_extract(payload, '$.context.recommended_strategy') as recommended_strategy,
    json_extract(payload, '$.context.supporting_risk') as risk_level,
    json_extract(payload, '$.context.estimated_token_usage') as estimated_tokens
FROM ingestion_observations
WHERE json_extract(payload, '$.context') IS NOT NULL
LIMIT 10
"""
result7 = pd.read_sql_query(query7, conn)
print("📊 Query 7: Context window assessments")
if not result7.empty:
    print(result7.to_string(index=False))
else:
    print("No context assessments found")

conn.close()


📊 Query 7: Context window assessments
                     document_id recommended_strategy risk_level  estimated_tokens
3f857ca539ef4595ae6b2775c23f07ed             document        low               242
2a6b68c8b120469ea6cb21707ecbfeaf             document        low                92
eac54e99f2a94080964eafd9c6af1902             document        low                83


## Step 6: Analyze Service Event Payloads

Service events contain detailed LLM metrics. Let's extract cost and latency data.


In [11]:
conn = sqlite3.connect(str(ANALYTICS_DB_PATH))

# Query 8: Extract LLM cost and token usage from event payloads
query8 = """
SELECT 
    service,
    name,
    timestamp,
    json_extract(payload, '$.model') as model,
    json_extract(payload, '$.total_tokens') as total_tokens,
    json_extract(payload, '$.input_tokens') as input_tokens,
    json_extract(payload, '$.output_tokens') as output_tokens,
    json_extract(payload, '$.cost') as cost,
    json_extract(payload, '$.latency_ms') as latency_ms
FROM ingestion_observation_events
WHERE json_extract(payload, '$.total_tokens') IS NOT NULL
ORDER BY timestamp DESC
LIMIT 20
"""
result8 = pd.read_sql_query(query8, conn)
print("📊 Query 8: LLM usage metrics from events")
if not result8.empty:
    print(result8.to_string(index=False))
    
    # Calculate totals
    if 'cost' in result8.columns and result8['cost'].notna().any():
        total_cost = result8['cost'].sum()
        total_tokens = result8['total_tokens'].sum()
        print(f"\n💰 Total cost: ${total_cost:.4f}")
        print(f"🔢 Total tokens: {total_tokens:,}")
else:
    print("No LLM metrics found in events")

conn.close()


📊 Query 8: LLM usage metrics from events
No LLM metrics found in events


In [12]:
conn = sqlite3.connect(str(ANALYTICS_DB_PATH))

# Query 9: Aggregate LLM costs by service and model
query9 = """
SELECT 
    service,
    json_extract(payload, '$.model') as model,
    COUNT(*) as call_count,
    SUM(json_extract(payload, '$.total_tokens')) as total_tokens,
    SUM(json_extract(payload, '$.cost')) as total_cost,
    AVG(json_extract(payload, '$.latency_ms')) as avg_latency_ms
FROM ingestion_observation_events
WHERE json_extract(payload, '$.cost') IS NOT NULL
GROUP BY service, model
ORDER BY total_cost DESC
"""
result9 = pd.read_sql_query(query9, conn)
print("📊 Query 9: LLM costs aggregated by service and model")
if not result9.empty:
    print(result9.to_string(index=False))
else:
    print("No cost data found")

conn.close()


📊 Query 9: LLM costs aggregated by service and model
No cost data found


## Step 7: Using Analytics Views

The analytics views module provides pre-built aggregations and summaries.


In [13]:
# Corpus overview - aggregate statistics
overview = corpus_overview(store)
print("📊 Corpus Overview:")
pprint(overview)


📊 Corpus Overview:
{'avg_chunks': 2,
 'avg_tokens': 139,
 'context_strategy_counts': {'document': 3},
 'documents': 3,
 'risk_counts': {'low': 3},
 'total_tokens': 417}


In [14]:
# Document size table - per-document metrics
doc_table = document_size_table(store)
if doc_table:
    df_docs = pd.DataFrame(doc_table)
    print("📊 Document Size Table:")
    print(df_docs.to_string(index=False))
else:
    print("No documents found")


📊 Document Size Table:
                     document_id                                                                                           source_uri  token_count  chunk_count schema_version context_strategy risk_level
eac54e99f2a94080964eafd9c6af1902 C:\Users\danie\Repos\spindle\spindle\notebooks\example_project\documents\doc3_techcorp_financials.md           83            1          1.0.0         document        low
2a6b68c8b120469ea6cb21707ecbfeaf  C:\Users\danie\Repos\spindle\spindle\notebooks\example_project\documents\doc2_techcorp_expansion.md           92            1          1.0.0         document        low
3f857ca539ef4595ae6b2775c23f07ed   C:\Users\danie\Repos\spindle\spindle\notebooks\example_project\documents\doc1_techcorp_overview.md          242            4          1.0.0         document        low


In [15]:
# Ontology recommendation metrics
ontology_metrics = ontology_recommendation_metrics(store)
print("📊 Ontology Recommendation Metrics:")
pprint(ontology_metrics)


📊 Ontology Recommendation Metrics:
{'avg_latency_ms': 47462.64520008117,
 'by_model': {'unknown': {'avg_latency_ms': 47462.64520008117,
                          'calls': 1,
                          'input_tokens': 2562,
                          'output_tokens': 3967,
                          'total_cost': 0.0,
                          'total_tokens': 0}},
 'by_scope': {'balanced': {'avg_latency_ms': 47462.64520008117,
                           'calls': 1,
                           'input_tokens': 2562,
                           'output_tokens': 3967,
                           'total_cost': 0.0,
                           'total_tokens': 0}},
 'total_calls': 1,
 'total_cost': 0.0,
 'total_tokens': 0}


In [16]:
# Triple extraction metrics
triple_metrics = triple_extraction_metrics(store)
print("📊 Triple Extraction Metrics:")
pprint(triple_metrics)


📊 Triple Extraction Metrics:
{'avg_latency_ms': 42010.083333123475,
 'by_model': {'unknown': {'avg_latency_ms': 42010.083333123475,
                          'avg_triples_per_call': 5.0,
                          'calls': 3,
                          'entities': 0,
                          'input_tokens': 10598,
                          'output_tokens': 10320,
                          'total_cost': 0.0,
                          'total_tokens': 0,
                          'triples': 15}},
 'by_scope': {'balanced': {'avg_latency_ms': 42010.083333123475,
                           'avg_triples_per_call': 5.0,
                           'calls': 3,
                           'entities': 0,
                           'input_tokens': 10598,
                           'output_tokens': 10320,
                           'total_cost': 0.0,
                           'total_tokens': 0,
                           'triples': 15}},
 'total_calls': 3,
 'total_cost': 0.0,
 'total_tokens': 0,
 'to

In [17]:
# Entity resolution metrics
resolution_metrics = entity_resolution_metrics(store)
print("📊 Entity Resolution Metrics:")
pprint(resolution_metrics)


📊 Entity Resolution Metrics:
{'edge_matching': {'avg_latency_ms': 10120.650547090918,
                   'by_model': {'unknown': {'avg_latency_ms': 10120.650547090918,
                                            'calls': 17,
                                            'input_tokens': 14690,
                                            'output_tokens': 8980,
                                            'total_cost': 0.0,
                                            'total_tokens': 0}},
                   'calls': 17,
                   'input_tokens': 14690,
                   'output_tokens': 8980,
                   'total_cost': 0.0,
                   'total_tokens': 0},
 'entity_matching': {'avg_latency_ms': 14144.086760096252,
                     'by_model': {'unknown': {'avg_latency_ms': 14144.086760096252,
                                              'calls': 5,
                                              'input_tokens': 5457,
                                              'outp

In [18]:
conn = sqlite3.connect(str(ANALYTICS_DB_PATH))

# Query 10: Documents with highest token counts and their associated events
query10 = """
SELECT 
    o.document_id,
    json_extract(o.payload, '$.structural.token_count') as token_count,
    json_extract(o.payload, '$.structural.chunk_count') as chunk_count,
    COUNT(e.id) as event_count,
    SUM(json_extract(e.payload, '$.cost')) as total_cost
FROM ingestion_observations o
LEFT JOIN ingestion_observation_events e ON o.id = e.observation_id
GROUP BY o.id, o.document_id
ORDER BY token_count DESC
LIMIT 10
"""
result10 = pd.read_sql_query(query10, conn)
print("📊 Query 10: Documents ranked by token count with costs")
if not result10.empty:
    print(result10.to_string(index=False))
else:
    print("No data found")

conn.close()


📊 Query 10: Documents ranked by token count with costs
                     document_id  token_count  chunk_count  event_count total_cost
3f857ca539ef4595ae6b2775c23f07ed          242            4            1       None
2a6b68c8b120469ea6cb21707ecbfeaf           92            1            1       None
eac54e99f2a94080964eafd9c6af1902           83            1            1       None


In [19]:
conn = sqlite3.connect(str(ANALYTICS_DB_PATH))

# Query 11: Time-based analysis - events per day
query11 = """
SELECT 
    DATE(timestamp) as date,
    COUNT(*) as event_count,
    COUNT(DISTINCT service) as unique_services,
    SUM(json_extract(payload, '$.cost')) as daily_cost
FROM ingestion_observation_events
GROUP BY DATE(timestamp)
ORDER BY date DESC
"""
result11 = pd.read_sql_query(query11, conn)
print("📊 Query 11: Daily event statistics")
if not result11.empty:
    print(result11.to_string(index=False))
else:
    print("No events found")

conn.close()


📊 Query 11: Daily event statistics
      date  event_count  unique_services daily_cost
2025-11-14            3                1       None


## Step 9: Export Data for Further Analysis

Export query results to CSV or work with them in pandas DataFrames.


In [20]:
conn = sqlite3.connect(str(ANALYTICS_DB_PATH))

# Create a comprehensive summary DataFrame
summary_query = """
SELECT 
    o.document_id,
    o.ingested_at,
    json_extract(o.payload, '$.structural.token_count') as token_count,
    json_extract(o.payload, '$.structural.chunk_count') as chunk_count,
    json_extract(o.payload, '$.metadata.source_uri') as source_uri,
    json_extract(o.payload, '$.context.recommended_strategy') as context_strategy,
    COUNT(e.id) as event_count,
    SUM(json_extract(e.payload, '$.cost')) as total_cost,
    AVG(json_extract(e.payload, '$.latency_ms')) as avg_latency_ms
FROM ingestion_observations o
LEFT JOIN ingestion_observation_events e ON o.id = e.observation_id
GROUP BY o.id
ORDER BY o.ingested_at DESC
"""
df_summary = pd.read_sql_query(summary_query, conn)
conn.close()

print("📊 Comprehensive Summary DataFrame:")
print(f"Shape: {df_summary.shape}")
print(f"\nFirst few rows:")
print(df_summary.head().to_string(index=False))

# Display basic statistics
if not df_summary.empty:
    print(f"\n📈 Statistics:")
    print(f"  Total documents: {len(df_summary)}")
    if 'token_count' in df_summary.columns and df_summary['token_count'].notna().any():
        print(f"  Total tokens: {df_summary['token_count'].sum():,}")
        print(f"  Average tokens per document: {df_summary['token_count'].mean():.0f}")
    if 'total_cost' in df_summary.columns and df_summary['total_cost'].notna().any():
        print(f"  Total cost: ${df_summary['total_cost'].sum():.4f}")
    
    # Optionally export to CSV
    # df_summary.to_csv('analytics_summary.csv', index=False)
    # print("\n✅ Exported to analytics_summary.csv")


📊 Comprehensive Summary DataFrame:
Shape: (3, 9)

First few rows:
                     document_id                ingested_at  token_count  chunk_count                                                                                           source_uri context_strategy  event_count total_cost avg_latency_ms
eac54e99f2a94080964eafd9c6af1902 2025-11-14 22:18:11.425474           83            1 C:\Users\danie\Repos\spindle\spindle\notebooks\example_project\documents\doc3_techcorp_financials.md         document            1       None           None
2a6b68c8b120469ea6cb21707ecbfeaf 2025-11-14 22:18:11.327198           92            1  C:\Users\danie\Repos\spindle\spindle\notebooks\example_project\documents\doc2_techcorp_expansion.md         document            1       None           None
3f857ca539ef4595ae6b2775c23f07ed 2025-11-14 22:18:11.233879          242            4   C:\Users\danie\Repos\spindle\spindle\notebooks\example_project\documents\doc1_techcorp_overview.md         document  

## Summary

This notebook demonstrated:

✅ **Database Connection**: Connected to analytics SQLite database  
✅ **Schema Exploration**: Examined table structures  
✅ **API Queries**: Used AnalyticsStore methods to fetch observations and events  
✅ **Direct SQL**: Performed SQL queries for detailed analysis  
✅ **JSON Extraction**: Extracted nested JSON data from payloads  
✅ **Aggregations**: Calculated statistics and summaries  
✅ **Analytics Views**: Used pre-built view functions  
✅ **Data Export**: Created DataFrames for further analysis  

### Key Tables

- `ingestion_observations`: Document observations with JSON payloads
- `ingestion_observation_events`: Service events linked to observations

### Common Query Patterns

1. **Count observations**: `SELECT COUNT(*) FROM ingestion_observations`
2. **Extract JSON fields**: `json_extract(payload, '$.structural.token_count')`
3. **Join observations with events**: `LEFT JOIN ingestion_observation_events`
4. **Aggregate by service**: `GROUP BY service`
5. **Time-based analysis**: `DATE(timestamp)` for daily aggregations

### Next Steps

- Explore specific document observations in detail
- Analyze LLM cost trends over time
- Compare metrics across different document types
- Build custom visualizations using pandas/matplotlib
- Integrate with the Streamlit dashboard for interactive exploration
